# Patat : Plateforme Agnostique de Traitement et d'Analyse des Textes

### Notebook 1 : Création du Recueil Complet
En partant du Recueil Brut issu de la google spreadsheet observateurs

### Synopsis
- Chargement et Nettoyage du Recueil
- Importation et Parsing des articles
- Nettoyage et Sauvegarde du Recueil Complet

### To do
- Moins de NaN dans le recueil
- Meilleure prise en compte des NaN dans la suite

# 0 - Préliminaires

In [1]:
import datetime
now = datetime.datetime.now()
file_date = now.strftime("%y%m%d")
#file_date='220824'
recueil_filename = f'data/corpus-work/{file_date}-RecueilBrut.csv'
recueil_complet_filename = f'data/corpus/{file_date}-RecueilComplet.csv'
print(recueil_filename)
print(recueil_complet_filename)

data/corpus-work/220919-RecueilBrut.csv
data/corpus/220919-RecueilComplet.csv


In [2]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [3]:
import importlib
import pandas as pd

In [4]:
import patat.imp.importer

## 1 - Chargement recueil
Le recueil a été exporté en csv de googlesheet.

Nom de fichier : **AAMMJJ-RecueilBrut.csv**

In [5]:
df_brut = pd.read_csv(recueil_filename)

In [6]:
df_brut

,# site,# article,Site,url,Auteur/trice,Date de publication,Titre,explication,infox,signé,...,invérifiable,propos rapporté,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné)
0,1,1,bonsens.org,https://bonsens.info/essais-pfizer-sur-le-vacc...,NaN,1er février 22,Essais Pfizer sur le vaccin anti-covid : le ra...,pseudo rapport par une pseudo scientifique mon...,oui,non,...,non,oui,oui,non,NaN,*,FB,vaccin,pseudo science,27 avril 22
1,2,2,breizh,https://www.breizh-info.com/2022/02/23/180457/...,NaN,23 février 22,Tests – vaccins – pass : fabrique de la paniqu...,yapa de pandémie et les vaccins ne marchent pas,oui,non,...,oui,non,non,oui,NaN,**,FB,vaccins,pseudo démonstration,mars 22
2,2,3,breizh,https://www.breizh-info.com/2022/02/20/180182/...,NaN,20 février 22,"Covid-19. La pandémie et les vaccins, pour mie...",yapa de pandémie et il y a un complot mondial,oui,oui,...,oui,oui,non,non,NaN,**,FB,vaccins,complot,mars 22
3,2,4,breizh,https://www.breizh-info.com/2022/04/19/185260/...,NaN,19 avril 2022,Covid-19. Une épidémie quasiment terminée avan...,démonstration que les vaccins n'ont servi à rien,oui,oui,...,oui,oui,non,non,NaN,**,FB,vaccins,complot,mars 22
4,2,5,breizh,https://www.breizh-info.com/2022/04/15/184984/...,NaN,15 avril 22,"Piscine, voile, ramadan : scènes de la vie ord...",absolument rien pour étayer cette histoire qui...,oui,non,...,oui,oui,non,non,NaN,*,FB,grand remplacement,histoire inventée,21 avril 22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,38,443,Wikistrike,https://www.wikistrike.com/2022/08/fresne-cett...,NaN,21 Aout 22,Fresnes: cette prison où les condamnés finisse...,NaN,non,non,...,non,oui,oui,non,non,NaN,CN,NaN,NaN,NaN
443,38,444,Wikistrike,https://www.wikistrike.com/2022/08/le-forum-ec...,NaN,23 Aout 22,Le Forum économique mondial estime qu’il exist...,Convaincre que le Forum economique mondial veu...,oui,non,...,non,oui,oui,non,oui,*,CN,NaN,NaN,NaN
444,38,445,Wikistrike,https://www.wikistrike.com/2022/08/le-masque-r...,NaN,24 Aout 22,"Le masque réduit les infections COVID de 0,06%",L'article prend une etude qui est loin d'etre ...,oui,non,...,non,non,non,non,non,**,CN,NaN,NaN,NaN
445,38,446,Wikistrike,https://www.wikistrike.com/2022/08/l-un-des-pl...,NaN,24 Aout 22,L’un des plus grands complexes mégalithiques d...,NaN,non,non,...,non,non,oui,non,non,NaN,CN,NaN,NaN,NaN


In [7]:
# Suppression des urls vides
df_brut = df_brut[df_brut['url'].notna()]

In [8]:
df_brut.keys()

Index(['# site', '# article', 'Site', 'url', 'Auteur/trice',
       'Date de publication', 'Titre', 'explication', 'infox', 'signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
       'intention', 'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
       'dénigrement', 'degré infox', 'observateur', 'thème (abandonné)',
       'type infox (abandonné)', 'Date de collection (abandonné)'],
      dtype='object')

In [9]:
labels = [
    'infox', 'signé',
    'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
    'points de vue différents présentés', 'écriture de qualité',
    'faits', 'opinions', 'fausse nouvelle', 'insinuations',
    'scientifique sulfureux', 'des pages et des pages',
    'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
    'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
]

In [10]:
def normalize_label(l):
    if l=='oui':
        return 1.0
#    elif l=='non':
#        return 0.0
    else:
        return 0.0

In [11]:
for label in labels:
    df_brut.loc[:,label]=df_brut[label].apply(normalize_label)

In [12]:
for label in labels:
    df_brut = df_brut[df_brut[label].notna()]

In [13]:
for label in labels:
    print(label,df_brut[df_brut[label].isna()].shape)

infox (0, 35)
signé (0, 35)
lieux, dates et personnes  (0, 35)
lieux, dates et personnes cohérentes (0, 35)
points de vue différents présentés (0, 35)
écriture de qualité (0, 35)
faits (0, 35)
opinions (0, 35)
fausse nouvelle (0, 35)
insinuations (0, 35)
scientifique sulfureux (0, 35)
des pages et des pages (0, 35)
des chiffres et des tableaux (0, 35)
sophisme (0, 35)
inversion de la preuve (0, 35)
invérifiable (0, 35)
propos rapporté (0, 35)
sources citées (0, 35)
titre décalé (0, 35)


### Extraction des sites

In [14]:
import urllib
def get_site(url):
    if type(url)==str:
        return urllib.parse.urlsplit(url).netloc

In [15]:
df_brut.loc[:,'site'] = df_brut['url'].apply(get_site)

In [16]:
df_brut

,# site,# article,Site,url,Auteur/trice,Date de publication,Titre,explication,infox,signé,...,propos rapporté,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site
0,1,1,bonsens.org,https://bonsens.info/essais-pfizer-sur-le-vacc...,NaN,1er février 22,Essais Pfizer sur le vaccin anti-covid : le ra...,pseudo rapport par une pseudo scientifique mon...,1.0,0.0,...,1.0,1.0,0.0,NaN,*,FB,vaccin,pseudo science,27 avril 22,bonsens.info
1,2,2,breizh,https://www.breizh-info.com/2022/02/23/180457/...,NaN,23 février 22,Tests – vaccins – pass : fabrique de la paniqu...,yapa de pandémie et les vaccins ne marchent pas,1.0,0.0,...,0.0,0.0,1.0,NaN,**,FB,vaccins,pseudo démonstration,mars 22,www.breizh-info.com
2,2,3,breizh,https://www.breizh-info.com/2022/02/20/180182/...,NaN,20 février 22,"Covid-19. La pandémie et les vaccins, pour mie...",yapa de pandémie et il y a un complot mondial,1.0,1.0,...,1.0,0.0,0.0,NaN,**,FB,vaccins,complot,mars 22,www.breizh-info.com
3,2,4,breizh,https://www.breizh-info.com/2022/04/19/185260/...,NaN,19 avril 2022,Covid-19. Une épidémie quasiment terminée avan...,démonstration que les vaccins n'ont servi à rien,1.0,1.0,...,1.0,0.0,0.0,NaN,**,FB,vaccins,complot,mars 22,www.breizh-info.com
4,2,5,breizh,https://www.breizh-info.com/2022/04/15/184984/...,NaN,15 avril 22,"Piscine, voile, ramadan : scènes de la vie ord...",absolument rien pour étayer cette histoire qui...,1.0,0.0,...,1.0,0.0,0.0,NaN,*,FB,grand remplacement,histoire inventée,21 avril 22,www.breizh-info.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,38,443,Wikistrike,https://www.wikistrike.com/2022/08/fresne-cett...,NaN,21 Aout 22,Fresnes: cette prison où les condamnés finisse...,NaN,0.0,0.0,...,1.0,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com
443,38,444,Wikistrike,https://www.wikistrike.com/2022/08/le-forum-ec...,NaN,23 Aout 22,Le Forum économique mondial estime qu’il exist...,Convaincre que le Forum economique mondial veu...,1.0,0.0,...,1.0,1.0,0.0,oui,*,CN,NaN,NaN,NaN,www.wikistrike.com
444,38,445,Wikistrike,https://www.wikistrike.com/2022/08/le-masque-r...,NaN,24 Aout 22,"Le masque réduit les infections COVID de 0,06%",L'article prend une etude qui est loin d'etre ...,1.0,0.0,...,0.0,0.0,0.0,non,**,CN,NaN,NaN,NaN,www.wikistrike.com
445,38,446,Wikistrike,https://www.wikistrike.com/2022/08/l-un-des-pl...,NaN,24 Aout 22,L’un des plus grands complexes mégalithiques d...,NaN,0.0,0.0,...,0.0,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com


## 2 - Acquisition et Parsing Articles

In [17]:
importlib.reload(patat.imp.importer)
imp = patat.imp.importer.Importer()

In [18]:
df_brut

,# site,# article,Site,url,Auteur/trice,Date de publication,Titre,explication,infox,signé,...,propos rapporté,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site
0,1,1,bonsens.org,https://bonsens.info/essais-pfizer-sur-le-vacc...,NaN,1er février 22,Essais Pfizer sur le vaccin anti-covid : le ra...,pseudo rapport par une pseudo scientifique mon...,1.0,0.0,...,1.0,1.0,0.0,NaN,*,FB,vaccin,pseudo science,27 avril 22,bonsens.info
1,2,2,breizh,https://www.breizh-info.com/2022/02/23/180457/...,NaN,23 février 22,Tests – vaccins – pass : fabrique de la paniqu...,yapa de pandémie et les vaccins ne marchent pas,1.0,0.0,...,0.0,0.0,1.0,NaN,**,FB,vaccins,pseudo démonstration,mars 22,www.breizh-info.com
2,2,3,breizh,https://www.breizh-info.com/2022/02/20/180182/...,NaN,20 février 22,"Covid-19. La pandémie et les vaccins, pour mie...",yapa de pandémie et il y a un complot mondial,1.0,1.0,...,1.0,0.0,0.0,NaN,**,FB,vaccins,complot,mars 22,www.breizh-info.com
3,2,4,breizh,https://www.breizh-info.com/2022/04/19/185260/...,NaN,19 avril 2022,Covid-19. Une épidémie quasiment terminée avan...,démonstration que les vaccins n'ont servi à rien,1.0,1.0,...,1.0,0.0,0.0,NaN,**,FB,vaccins,complot,mars 22,www.breizh-info.com
4,2,5,breizh,https://www.breizh-info.com/2022/04/15/184984/...,NaN,15 avril 22,"Piscine, voile, ramadan : scènes de la vie ord...",absolument rien pour étayer cette histoire qui...,1.0,0.0,...,1.0,0.0,0.0,NaN,*,FB,grand remplacement,histoire inventée,21 avril 22,www.breizh-info.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,38,443,Wikistrike,https://www.wikistrike.com/2022/08/fresne-cett...,NaN,21 Aout 22,Fresnes: cette prison où les condamnés finisse...,NaN,0.0,0.0,...,1.0,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com
443,38,444,Wikistrike,https://www.wikistrike.com/2022/08/le-forum-ec...,NaN,23 Aout 22,Le Forum économique mondial estime qu’il exist...,Convaincre que le Forum economique mondial veu...,1.0,0.0,...,1.0,1.0,0.0,oui,*,CN,NaN,NaN,NaN,www.wikistrike.com
444,38,445,Wikistrike,https://www.wikistrike.com/2022/08/le-masque-r...,NaN,24 Aout 22,"Le masque réduit les infections COVID de 0,06%",L'article prend une etude qui est loin d'etre ...,1.0,0.0,...,0.0,0.0,0.0,non,**,CN,NaN,NaN,NaN,www.wikistrike.com
445,38,446,Wikistrike,https://www.wikistrike.com/2022/08/l-un-des-pl...,NaN,24 Aout 22,L’un des plus grands complexes mégalithiques d...,NaN,0.0,0.0,...,0.0,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com


In [19]:
df_crawl = df_brut

In [20]:
def crawl_item(url):
    html = ''
    print(url)
    try:
        html = imp.get_webpage(url, use_cache=True)
    except:
        print('Exception :' + str(url))
    return html

In [21]:
df_crawl['html']=df_crawl['url'].apply(crawl_item)

https://bonsens.info/essais-pfizer-sur-le-vaccin-anti-covid-le-rapport-explosif-de-christine-cotton/
https://www.breizh-info.com/2022/02/23/180457/tests-vaccins-pass-fabrique-de-la-panique-braquage-des-comptes-publics/
https://www.breizh-info.com/2022/02/20/180182/covid-19-la-pandemie-et-les-vaccins-pour-mieux-justifier-un-coup-detat-mondial/
https://www.breizh-info.com/2022/04/19/185260/covid-19-une-epidemie-quasiment-terminee-avant-lete-au-niveau-mondial/
https://www.breizh-info.com/2022/04/15/184984/piscine-voile-ramadan-scenes-de-la-vie-ordinaire-dans-un-prestigieux-lycee-breton/
https://www.breizh-info.com/2021/11/02/173738/faut-il-vous-metisser-comme-le-suggere-la-presse-mainstream/
https://www.breizh-info.com/2021/12/20/176485/qi-races-differences-intelligence-delinquance-wokisme-tour-dhorizon-du-negationnisme-biologique/
https://www.breizh-info.com/2022/05/15/186521/geopolitique-une-analyse-pro-russe-de-la-strategie-militaire-en-ukraine/
https://www.breizh-info.com/2022/05/18/1

https://lemediaen442.fr/mathilde-panot-accusee-dantisemitisme-pour-avoir-prononce-le-mot-rescapee/
https://lemediaen442.fr/les-sanctions-contre-la-russie-provoquent-leffondrement-de-leuro-le-ministre-bruno-le-maire-sest-totalement-trompe/
https://lemediaen442.fr/dictature-acte-1-vote-de-la-loi-de-veille-sanitaire-a-lassemblee/
https://lemediaen442.fr/pays-bas-le-gouvernement-veut-reduire-le-cheptel-dun-tiers-le-pays-est-au-bord-de-la-paralysie-complete/
https://lemediaen442.fr/marcel-d-vous-explique-en-4-4-2-qui-est-reellement-vladimir-poutine/
https://lemediaen442.fr/le-passe-sanitaire-revient-pour-les-vacances-non-vaccines-et-retifs-aux-tests-seront-bien-emmerdes/
https://lemediaen442.fr/le-ministere-de-la-sante-allemand-annonce-1-effet-secondaire-grave-suspecte-toutes-les-5-000-doses-de-vaccin/
https://lemediaen442.fr/pendant-la-penurie-de-cereales-les-affaires-continuent/
https://lemediaen442.fr/la-russie-nest-pas-isolee-un-important-sommet-sest-tenu-en-presence-de-poutine-avec-rai

ERROR:root:Importer.get_webpage:https://lesmoutonsrebelles.com/lhomme-qui-a-vendu-lukraine/
ERROR:root:Importer.get_webpage:https://www.nouvelordremondial.cc/2022/09/16/immortalite-quantique-je-suis-morte-mais-je-me-suis-reveillee-dans-une-realite-differente/


https://lemediaen442.fr/le-president-ukrainien-zelensky-se-cache-dans-lambassade-des-etats-unis-en-pologne-selon-un-depute-dukraine/
https://lesmoutonsrebelles.com/une-etrange-structure-cubique-photographiee-sur-la-face-cachee-de-la-lune/
https://lesmoutonsrebelles.com/surmortalite-post-vaccinale-des-scientifiques-allemands-sondent-les-statistiques-officielles/
https://lesmoutonsrebelles.com/emmanuel-macron-est-un-psychopathe-amoral-selon-le-psychiatre-italien-adriano-segatori/
https://lesmoutonsrebelles.com/biden-utilise-cyniquement-lukraine-pour-couvrir-le-sabotage-alimentaire/
https://lesmoutonsrebelles.com/george-soros-force-les-grandes-entreprises-a-boycotter-twitter-pour-contrer-elon-musk/
https://lesmoutonsrebelles.com/igor-kirillov-general-russe-les-etats-unis-et-lotan-preparent-une-attaque-chimique-ou-nucleaire-pour-accuser-la-russie/
https://lesmoutonsrebelles.com/la-fin-irresistible-dun-empire-predateur/
https://lesmoutonsrebelles.com/la-variole-du-singe-trompe-moi-deux-fois

In [22]:
import patat.imp.parser_params
import patat.imp.parser

In [23]:
importlib.reload(patat.imp.parser_params)
importlib.reload(patat.imp.parser)
parser = patat.imp.parser.Parser()

In [24]:
def parse_article(url,html,site):
    record = {'url':url,'title':'','article':'','words':0}
    if html:
        if parser.is_known_site(url):
            record = parser.parse_article(url,html,site)
            record['parse_method']='parse'
        else:
            record = parser.parse_raw_article(url,html)
            record['parse_method']='raw'
    
    return record

In [25]:
%%time
# Attention ! Le parsing peut prendre du temps
parsed = df_crawl.apply(lambda row: parse_article(row['url'], row['html'], row['site']), axis=1)

CPU times: user 20.6 s, sys: 268 ms, total: 20.9 s
Wall time: 20.9 s


In [26]:
df_parsed = pd.DataFrame(list(parsed))

## 3 - Vérification du dataframe parsé

In [50]:
len(df_parsed)

447

## 4 - Création Recueil Complet
Le recueil complet est exporté en csv.

Nom de fichier : **AAMMJJ-RecueilComplet.csv**

In [28]:
df_complet = pd.merge(df_parsed,df_brut)

In [40]:
df_complet[df_complet['url'].isna()]

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site,text


In [30]:
#df_parsed=df_parsed.rename(columns={'url':'url_parsed'})
#df_complet=pd.concat([df_parsed,df_brut],axis=1)

In [31]:
#df_complet['url_parsed']

In [32]:
#df_complet[df_complet['url']!=df_complet['url_parsed']]

In [33]:
# Articles vides
df_complet[df_complet['article']=='']

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site,html
309,https://lesmoutonsrebelles.com/lhomme-qui-a-ve...,,,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,NaN,**,FB,guerre,NaN,NaN,lesmoutonsrebelles.com,
351,https://www.nouvelordremondial.cc/2022/09/16/i...,,,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,non,*,FB,NaN,NaN,NaN,www.nouvelordremondial.cc,


In [34]:
# Suppression des articles vides
df_complet = df_complet[df_complet['article']!='']

In [35]:
# Titres vides
df_complet[df_complet['title']=='']

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site,html
385,http://www.reopen911.info/11-septembre/une-att...,,AccueilBlogVidéosPoints-clésTémoignagesProfess...,raw,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,non,*,CN,NaN,NaN,NaN,www.reopen911.info,"﻿<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 ..."


In [36]:
# Suppression des titres vides
df_complet = df_complet[df_complet['title']!='']

In [37]:
df_complet.drop('html',axis=1,inplace=True)

In [38]:
# Concaténation Titre Abstract Article
df_complet.loc[:,'text'] = df_complet['title']+'\n'+df_complet['article']

In [47]:
df_complet.to_csv(recueil_complet_filename,index=False)

# Bricolages

In [48]:
df_complet.loc[445:]

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site,text
445,https://www.wikistrike.com/2022/08/etude-22-de...,Étude: 22 % des moins de 50 ans diagnostiqués ...,"Le 27 juillet 2022, l'organisation ""Children's...",parse,Publié par wikistrike.com\n ...,[],,,2022-08-04T00:00:00,NaN,...,0.0,0.0,NaN,*,FB,NaN,NaN,NaN,www.wikistrike.com,Étude: 22 % des moins de 50 ans diagnostiqués ...
446,https://www.wikistrike.com/2022/08/la-cabale-m...,La cabale mondialiste promeut un régime à base...,Le Forum économique mondial envisage un systèm...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-07T00:00:00,NaN,...,0.0,1.0,NaN,*,FB,NaN,NaN,NaN,www.wikistrike.com,La cabale mondialiste promeut un régime à base...
447,https://www.wikistrike.com/2022/08/compteur-li...,Compteur Linky : les personnes électrosensible...,Après avoir installé puis refusé de retirer de...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-17T00:00:00,NaN,...,0.0,0.0,non,NaN,FB,NaN,NaN,NaN,www.wikistrike.com,Compteur Linky : les personnes électrosensible...
448,https://www.wikistrike.com/2022/08/washington-...,Washington enverrait des mercenaires afghans c...,Des spécialistes originaires d'Afghanistan par...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-20T00:00:00,NaN,...,0.0,0.0,non,**,CN,NaN,NaN,NaN,www.wikistrike.com,Washington enverrait des mercenaires afghans c...
449,https://www.wikistrike.com/2022/08/les-vaccine...,Les vaccinés infectés au Covid restent contagi...,Une étude publiée le 21 juin 2022 par le New E...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-20T00:00:00,NaN,...,1.0,0.0,oui,**,CN,NaN,NaN,NaN,www.wikistrike.com,Les vaccinés infectés au Covid restent contagi...
450,https://www.wikistrike.com/2022/08/fresne-cett...,Fresnes: cette prison où les condamnés finisse...,Cette photo est une erreur \n\nEt nul n'est au...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-21T00:00:00,NaN,...,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com,Fresnes: cette prison où les condamnés finisse...
451,https://www.wikistrike.com/2022/08/le-forum-ec...,Le Forum économique mondial estime qu’il exist...,Ne trahissant pas sa réputation de méchant d’u...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-23T00:00:00,NaN,...,1.0,0.0,oui,*,CN,NaN,NaN,NaN,www.wikistrike.com,Le Forum économique mondial estime qu’il exist...
452,https://www.wikistrike.com/2022/08/le-masque-r...,"Le masque réduit les infections COVID de 0,06%",par Collectif Reinfocovid.\nCet article illust...,parse,Publié par wikistrike.com\n ...,[],,,2022-08-24T00:00:00,NaN,...,0.0,0.0,non,**,CN,NaN,NaN,NaN,www.wikistrike.com,"Le masque réduit les infections COVID de 0,06%..."
453,https://www.wikistrike.com/2022/08/l-un-des-pl...,L’un des plus grands complexes mégalithiques d...,"En Espagne, le site d’une future plantation d’...",parse,Publié par wikistrike.com\n ...,[],,,2022-08-24T00:00:00,NaN,...,1.0,0.0,non,NaN,CN,NaN,NaN,NaN,www.wikistrike.com,L’un des plus grands complexes mégalithiques d...
454,https://www.wikistrike.com/2022/09/les-command...,Les commandos ukrainiens qui ont attaqué la ce...,(!) Source russe \n\n\nLes quelques unités de ...,parse,Publié par wikistrike.com\n ...,[],,,2022-09-02T00:00:00,NaN,...,1.0,0.0,oui,*,FB,NaN,NaN,NaN,www.wikistrike.com,Les commandos ukrainiens qui ont attaqué la ce...


In [ ]:
df_parsed.loc[:,'url']=df_parsed['url_parsed']

In [ ]:
pd.merge(df_parsed,df_brut)
#df_parsed.join(df_b,on='url')

In [ ]:
df_brut.keys()

In [ ]:
df_b = df_brut.loc[:,['url','explication', 'infox', 'signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
       'intention', 'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
       'dénigrement', 'degré infox', 'observateur', 'thème (abandonné)',
       'type infox (abandonné)', 'Date de collection (abandonné)']]

In [ ]:
import patat.app.df_viewer

In [ ]:
viewer = patat.app.df_viewer.DfViewer()

In [ ]:
tree = viewer.get_tree_page()

In [ ]:
tree['tree'][1]

In [ ]:
df_complet

In [ ]:
df_parsed['url'] = df_parsed['url_parsed']

In [ ]:
df_brut['url']

In [ ]:
df_complet['article'][1]

In [ ]:
parser.clean_article(df_complet['article'][1])